In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
# 1 if using cudnn on GPU else using CPU
CUDNN_GPU = 1
model_path = 'my_model.csv'

In [3]:
pd.read_csv(model_path)

,Unnamed: 0,Layer,Filter,kernel,Stride,Output
0,0,Input,NaN,NaN,NaN,224x224x3
1,1,Conv,16.0,3x3,1.0,224x224x16
2,2,MaxPooling,NaN,2x2,2.0,112x112x16
3,3,Conv,32.0,3x3,1.0,112x112x32
4,4,MaxPooling,NaN,2x2,2.0,56x56x32
5,5,Conv,64.0,3x3,1.0,56x56x64
6,6,MaxPooling,NaN,2x2,2.0,28x28x64
7,7,Conv,128.0,3x3,1.0,28x28x128
8,8,MaxPooling,NaN,2x2,2.0,14x14x128
9,9,Conv,256.0,3x3,1.0,14x14x256


In [4]:
def cnn_model(input, model_path, is_training=True):
    use_cudnn_on_gpu = True if CUDNN_GPU == 0 else False
    my_model = pd.read_csv(model_path)
    
    ### Define some function for making layers
    def make_input(input_, out_shape, name):
        result = tf.reshape(input_, [1,out_shape[0],out_shape[1],out_shape[2]], name=name)
        tf.summary.histogram(name, result)
        return result

    def make_conv(input_, filter_channel, filter_, strides, name):
        in_channel = tf.shape(input_)[-1]
        filter_ = list(map(int,filter_.split('x')))
        filter_ = tf.Variable(tf.random_normal([filter_[0],filter_[1],in_channel,int(filter_channel)]), 
                              name=name+'_filter')
        result = tf.nn.conv2d(input=input_, 
                         filter=filter_,
                         strides=[int(strides)]*4,
                         padding='SAME',
                         use_cudnn_on_gpu=use_cudnn_on_gpu,
                         name=name)
        tf.summary.histogram(name, result)
        return result
        
    def make_maxpool(input_, filter_, strides, name):
        in_channel = tf.shape(input_)[-1]
        filter_ = list(map(int,filter_.split('x')))
        ksize = tf.Variable(tf.random_normal([filter_[0],filter_[1],in_channel,in_channel]), name=name+'_filter')
        result = tf.nn.max_pool(value=input_,
                             ksize=ksize,
                             strides=[int(strides)]*4,
                             padding='SAME',
                             name=name)
        return result
        
    def make_flatten(input_, name):
        return tf.contrib.layers.flatten(inputs=input_, scope=name)
        
    def make_fc(input_, out_shape, name):
        result = tf.contrib.layers.fully_connected(inputs=input_,
                                                num_outputs=out_shape[0],
                                                scope=name)
        tf.summary.histogram(name, result)
        return result
        
    def make_reshape(input_, out_shape, name):
        result = tf.reshape(tensor=input_,
                         shape=out_shape,
                         name=name)
        tf.summary.histogram(name, result)
        return result
    
    ### Generate the model base on the model file
    output = input
    layer_match = {
            'Input':      lambda input_, params: make_input(input_, params[5], 'input_image'),
            'Conv':       lambda input_, params: make_conv(input_, params[2], params[3], params[4], 'layer_'+params[0]),
            'MaxPooling': lambda input_, params: make_maxpool(input_, params[3], params[4], 'layer_'+params[0]),
            'Flatten':    lambda input_, params: make_flatten(input_, 'layer_'+params[0]),
            'Fc':         lambda input_, params: make_fc(input_, params[5], 'layer_'+params[0]),
            'Reshape':    lambda input_, params: make_reshape(input_, params[5], 'layer_'+params[0])
            
        }
    for layer in my_model.values:
        # preprocessing layer input
        layer[-1] = np.array(list(map(int,layer[-1].split('x'))))
        # map layer
        output = layer_match[layer[1]](output, layer)
        
    return output